In [1]:
import requests
import time
import pymysql
from bs4 import BeautifulSoup
from selenium import webdriver

### API approach, has limits per day - not worth it, continue to full scraping approach

In [2]:
links = ['https://github.com/poloclub/cnn-explainer']
github = 'https://api.github.com/'
gituser = 'https://github-user.now.sh/?username='

def rest_get(apilink):
    print(apilink)
    resp = requests.get(apilink)
    if resp.status_code != 200:
        # This means something went wrong.
        raise Exception('GET {} {}'.format(apilink, resp.status_code))
    else:
        return resp
    
for l in links:
    repo_split = l.split('/')
    owner, repo = repo_split[-2], repo_split[-1]
    print('Requestiong: ',owner, repo)
    resp = rest_get(f'{github}repos/{owner}/{repo}/contributors')
    for user in resp.json():
        print(f'user "{user["login"]}" has {user["contributions"]} contributions')
        user_info = rest_get(gituser+user["login"])
        print(f"Name: {user_info['name']}, Description: {user_info['description']}")
    

Requestiong:  poloclub cnn-explainer
https://api.github.com/repos/poloclub/cnn-explainer/contributors
user "xiaohk" has 189 contributions
https://github-user.now.sh/?username=xiaohk


TypeError: 'Response' object is not subscriptable

### Getting list of repositories and authors

In [53]:
conn=pymysql.connect(host='mysql24.ezhostingserver.com', port=3306, user='krw',password='Y>!V@N_26@]cfJ7(')

cursor=conn.cursor()
data = []
cursor.execute("SELECT krw.repo.paper_url, repo_url, author \
               FROM krw.repo INNER JOIN krw.paper_author \
               ON krw.repo.paper_url=krw.paper_author.paper_url limit 5000;")#("SELECT * FROM krw.training_view limit 10;")
for r in cursor:
    data.append(list(r))

In [54]:
data

[['https://paperswithcode.com/paper/digging-into-self-supervised-monocular-depth-1',
  'https://github.com/nianticlabs/monodepth2',
  'Clement Godard'],
 ['https://paperswithcode.com/paper/digging-into-self-supervised-monocular-depth-1',
  'https://github.com/nianticlabs/monodepth2',
  'Oisin Mac Aodha'],
 ['https://paperswithcode.com/paper/digging-into-self-supervised-monocular-depth-1',
  'https://github.com/nianticlabs/monodepth2',
  'Michael Firman'],
 ['https://paperswithcode.com/paper/digging-into-self-supervised-monocular-depth-1',
  'https://github.com/nianticlabs/monodepth2',
  'Gabriel J. Brostow'],
 ['https://paperswithcode.com/paper/unsupervised-training-for-3d-morphable-model',
  'https://github.com/google/tf_mesh_renderer',
  'Kyle Genova'],
 ['https://paperswithcode.com/paper/unsupervised-training-for-3d-morphable-model',
  'https://github.com/google/tf_mesh_renderer',
  'Forrester Cole'],
 ['https://paperswithcode.com/paper/unsupervised-training-for-3d-morphable-model',

In [56]:
insert = data.pop(0)
# Version with name as one string
final_data = [[insert[0], insert[1], [insert[2]]]]
for d in data:
    if d[0]==final_data[-1][0]:
        final_data[-1][2].append(d[2])
    else:
        final_data.append([d[0], d[1], [d[2]]])   
print(len(final_data))
final_data[0:50]

1148


[['https://paperswithcode.com/paper/digging-into-self-supervised-monocular-depth-1',
  'https://github.com/nianticlabs/monodepth2',
  ['Oisin Mac Aodha', 'Michael Firman', 'Gabriel J. Brostow']],
 ['https://paperswithcode.com/paper/unsupervised-training-for-3d-morphable-model',
  'https://github.com/google/tf_mesh_renderer',
  ['Kyle Genova',
   'Forrester Cole',
   'Aaron Maschinot',
   'Aaron Sarna',
   'Daniel Vlasic',
   'William T. Freeman']],
 ['https://paperswithcode.com/paper/multi-source-domain-adaptation-and-semi',
  'https://github.com/Panda-Peter/visda2019-multisource',
  ['Yingwei Pan', 'Yehao Li', 'Qi Cai', 'Yang Chen', 'Ting Yao']],
 ['https://paperswithcode.com/paper/high-fidelity-speech-synthesis-with-1',
  'https://github.com/yanggeng1995/GAN-TTS',
  ['Mikołaj Bińkowski',
   'Jeff Donahue',
   'Sander Dieleman',
   'Aidan Clark',
   'Erich Elsen',
   'Norman Casagrande',
   'Luis C. Cobo',
   'Karen Simonyan']],
 ['https://paperswithcode.com/paper/adaptive-affinity-fi

In [27]:
cursor.execute("SHOW COLUMNS FROM krw.repo")#("SELECT * FROM krw.training_view limit 10;")
for r in cursor:
    print(list(r))

['iRepoID', 'int(11)', 'NO', 'PRI', None, 'auto_increment']
['paper_url', 'varchar(255)', 'YES', '', None, '']
['paper_title', 'varchar(255)', 'YES', '', None, '']
['paper_arxiv_id', 'varchar(255)', 'YES', '', None, '']
['paper_url_abs', 'varchar(255)', 'YES', '', None, '']
['paper_url_pdf', 'varchar(255)', 'YES', '', None, '']
['repo_url', 'varchar(255)', 'YES', '', None, '']
['mentioned_in_paper', 'varchar(255)', 'YES', '', '', '']
['mentioned_in_github', 'varchar(255)', 'YES', '', '', '']
['framework', 'varchar(255)', 'YES', '', None, '']


### Full scraping approach

In [61]:
paper_data = {}
driver = webdriver.Firefox()

for paper in final_data:
    repo = paper[1]
    print(repo, paper[2]) # github repo and authors
    driver.get(f'{repo}/graphs/contributors')
    time.sleep(6) # wait for contribution graph to load
    website = BeautifulSoup(driver.page_source, 'html')
    boxes = website.find_all('h3', {'class':'border-bottom p-2 lh-condensed'})
    for user in boxes[0:10]:   # take just top 10 contributors, some repos might have 100+ contributors
        username = user.find('a', {'class':'text-normal', 'data-hovercard-type':'user'}).text.strip()
        info_line = user.find_all(('span', {'class':'cmeta'}))
        
        scrape = rest_get(f'https://github.com/{username}')
        soup = BeautifulSoup(scrape.content, 'html.parser')
        name = soup.find('span', class_='p-name vcard-fullname d-block overflow-hidden').text.strip()
        bio = soup.find('div', class_='p-note user-profile-bio mb-2 js-user-profile-bio').text.strip()
        print(f'name: {name} and {type(name)}, bio: {bio}')
        
        if name!="" and name in paper[2]:
            if paper[0] not in paper_data:
                paper_data[paper[0]]= [paper[1], {}]
            #print(info_line[1])
            commits = info_line[1].find('a').text.strip().split()[0]
            added = info_line[1].find('span', {'class':"text-green text-normal"}).text.strip().split()[0]
            deleted = info_line[1].find('span', {'class':"text-red text-normal"}).text.strip().split()[0]
            paper_data[paper[0]][1][name] = [commits, added, deleted]
            print(f'User {username}: commits {commits} , changed lines + {added} - {deleted}')
    print()

https://github.com/nianticlabs/monodepth2 ['Oisin Mac Aodha', 'Michael Firman', 'Gabriel J. Brostow']
https://github.com/mdfirman
name:  and <class 'str'>, bio: 
https://github.com/mrharicot
name: Clément Godard and <class 'str'>, bio: 
https://github.com/daniyar-niantic
name: Daniyar Turmukhambetov and <class 'str'>, bio: Senior Research Scientist at Niantic

https://github.com/google/tf_mesh_renderer ['Kyle Genova', 'Forrester Cole', 'Aaron Maschinot', 'Aaron Sarna', 'Daniel Vlasic', 'William T. Freeman']
https://github.com/fcole
name:  and <class 'str'>, bio: 
https://github.com/kylegenova
name: Kyle Genova and <class 'str'>, bio: 
User kylegenova: commits 2 , changed lines + 18 - 1
https://github.com/TerisGH
name:  and <class 'str'>, bio: 

https://github.com/Panda-Peter/visda2019-multisource ['Yingwei Pan', 'Yehao Li', 'Qi Cai', 'Yang Chen', 'Ting Yao']
https://github.com/Panda-Peter
name:  and <class 'str'>, bio: Yingwei Pan

https://github.com/yanggeng1995/GAN-TTS ['Mikołaj Bińk


https://github.com/DylanWusee/pointconv_pytorch ['Charles R. Qi', 'Hao Su', 'Kaichun Mo', 'Leonidas J. Guibas']


WebDriverException: Message: Failed to decode response from marionette


In [47]:
paper_data

{'https://paperswithcode.com/paper/unsupervised-training-for-3d-morphable-model': ['https://github.com/google/tf_mesh_renderer',
  {'Kyle Genova': ['2', '18', '1']}],
 'https://paperswithcode.com/paper/adaptive-affinity-fields-for-semantic': ['https://github.com/twke18/Adaptive_Affinity_Fields',
  {'Tsung-Wei Ke': ['2', '24', '6'], 'Jyh-Jing Hwang': ['9', '31', '26']}],
 'https://paperswithcode.com/paper/improving-information-retrieval-results-for': ['https://github.com/adelra/query-expansion',
  {'Adel Rahimi': ['2', '99,652', '0']}],
 'https://paperswithcode.com/paper/the-double-sphere-camera-model': ['https://github.com/VladyslavUsenko/basalt-mirror',
  {'Vladyslav Usenko': ['227', '66,951', '10,162'],
   'Nikolaus Demmel': ['24', '662', '1,219']}],
 'https://paperswithcode.com/paper/the-tum-vi-benchmark-for-evaluating-visual': ['https://github.com/VladyslavUsenko/basalt-mirror',
  {'Vladyslav Usenko': ['227', '66,951', '10,162'],
   'Nikolaus Demmel': ['24', '662', '1,219']}],
 'ht

In [3]:
repos = ['https://github.com/poloclub/cnn-explainer', 'https://github.com/openai/jukebox']

In [4]:
driver = webdriver.Firefox()

for repo in repos:
    print(repo)
    driver.get(f'{repo}/graphs/contributors')
    time.sleep(2)
    #delay of some kind wait for load time.sleep(3) or selenium wait for an element to be visible
    website = BeautifulSoup(driver.page_source, 'html')
    boxes = website.find_all('h3', {'class':'border-bottom p-2 lh-condensed'})
    for user in boxes:
        username = user.find('a', {'class':'text-normal', 'data-hovercard-type':'user'}).text.strip()
        info_line = user.find_all(('span', {'class':'cmeta'}))
        #print(info_line[1])
        commits = info_line[1].find('a').text.strip().split()[0]
        added = info_line[1].find('span', {'class':"text-green text-normal"}).text.strip().split()[0]
        deleted = info_line[1].find('span', {'class':"text-red text-normal"}).text.strip().split()[0]
        print(f'User {username}: commits {commits} , changed lines + {added} - {deleted}')
        scrape = rest_get(f'https://github.com/{username}')
        soup = BeautifulSoup(scrape.content, 'html.parser')
        name = soup.find('span', class_='p-name vcard-fullname d-block overflow-hidden').text.strip()
        bio = soup.find('div', class_='p-note user-profile-bio mb-2 js-user-profile-bio').text.strip()
        print(f'name: {name}, bio: {bio}')
        print()

https://github.com/poloclub/cnn-explainer
User xiaohk: commits 189 , changed lines + 35,562 - 14,943
https://github.com/xiaohk
name: Jay Wang, bio: Machine Learning PhD student at Georgia Tech.


Life is not always 🦄 and 🌈, but my favorite emoji is 😉

User rturko3: commits 58 , changed lines + 2,870 - 950
https://github.com/rturko3
name: , bio: CS Undergraduate @ Georgia Tech

User oshaikh13: commits 8 , changed lines + 117 - 74
https://github.com/oshaikh13
name: Omar Shaikh, bio: no u

https://github.com/openai/jukebox
User prafullasd: commits 49 , changed lines + 54,045 - 301
https://github.com/prafullasd
name: Prafulla Dhariwal, bio: 

User heewooj: commits 24 , changed lines + 393 - 344
https://github.com/heewooj
name: Heewoo Jun, bio: 

User jongwook: commits 5 , changed lines + 18 - 13
https://github.com/jongwook
name: Jong Wook Kim, bio: 

User mcleavey: commits 4 , changed lines + 964 - 3
https://github.com/mcleavey
name: Christine Payne, bio: OpenAI Researcher. Stanford Master